# Treinamento

## Inicialização

In [2]:
# Config Inicial
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import shutil

from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor

from sklearn.metrics import *

from mlflow.tracking import MlflowClient
import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature

### MLFlow

In [3]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")

mlflow.set_experiment("ecd15")

<Experiment: artifact_location='/app/mlruns/0', creation_time=1744853753613, experiment_id='0', last_update_time=1744853767255, lifecycle_stage='active', name='ecd15', tags={}>

In [4]:
seeds = [42, 47, 123]

## Dataset

O pré-processamento inicial foi realizado no dataset `brasile-real-state-dataset.csv`, onde as colunas de latitude e longitude foram convertidas em informações de cidade e estado. Esse processo resultou na criação do dataset `brasil_estado_cidade.csv`, que está sendo utilizado para o treinamento dos modelos apresentados a seguir.


### Leitura

In [5]:
# Descomentar, quando quiser carregar para o treinamento, dados originais defasados para primeira execução do treinamento
#shutil.copyfile("dataset/brasil_estado_cidade.csv", "dataset/dados_treinamento.csv")


# Carregar o conjunto de dados
dados = pd.read_csv("dataset/dados_treinamento.csv", encoding="utf-8")

# Eliminando registros com valores null
dados.dropna(inplace=True)

# float64
dados = dados.astype({col: 'float64' for col in dados.select_dtypes(include='int').columns})

def remover_outliers_por_cidade(df):
    # Remove outliers da coluna 'price_brl' agrupando por cidade.

    def remover_outliers_grupo(grupo):
        # Remove outliers de um grupo usando o método Z-score.
        z_scores = np.abs(stats.zscore(grupo["price_brl"]))
        return grupo[(z_scores < 3)]

    df_filtrado = df.groupby("city").apply(remover_outliers_grupo).reset_index(drop=True)
    return df_filtrado

dados = remover_outliers_por_cidade(dados)

dados.sample(10)

/tmp/ipykernel_149/445335293.py:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_filtrado = df.groupby("city").apply(remover_outliers_grupo).reset_index(drop=True)


,id,property_type,state,region,lat,lon,area_m2,price_brl,city
4108,1038.0,apartment,Rio Grande do Sul,South,-30.105619,-51.242771,69.0,332156.85,Porto Alegre
10898,12709.0,apartment,So Paulo,Southeast,-23.553312,-46.564880,185.0,1387397.48,São Paulo
10384,11771.0,house,So Paulo,Southeast,-23.653430,-46.697710,130.0,978165.11,São Paulo
6268,316.0,apartment,Pernambuco,Northeast,-8.089291,-34.903793,170.0,1247659.60,Recife
2180,8974.0,apartment,Santa Catarina,South,-27.591585,-48.497643,106.0,748595.73,Florianópolis
1149,9435.0,apartment,Santa Catarina,South,-26.985996,-48.636383,170.0,698689.35,Balneário Camboriú
3664,1173.0,apartment,Rio Grande do Sul,South,-29.919458,-50.224442,68.0,239550.63,Osório
3669,7026.0,apartment,Santa Catarina,South,-27.767431,-48.631371,106.0,557734.76,Palhoça
4134,1080.0,apartment,Rio Grande do Sul,South,-30.019997,-51.170139,98.0,449157.45,Porto Alegre
7005,4724.0,apartment,Rio de Janeiro,Southeast,-22.972993,-43.393232,70.0,488084.41,Rio de Janeiro


### Tratamento Features

In [6]:
# Exemplo de pré-processamento
x_features = dados.drop(["price_brl"], axis=1)  # Features
y_target = dados["price_brl"]  # Variável alvo

# Identificando colunas numéricas e categóricas
numeric_features = x_features.select_dtypes(include=['number']).columns
categorical_features = x_features.select_dtypes(include=['object']).columns

# Criando transformadores
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Criando o ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])


### Divisão dos Dados

In [7]:
test_sizes = [0.1, 0.2, 0.3]
#x_train, x_test, y_train, y_test = train_test_split(x_features, y_target, test_size=0.2, random_state=42)

## Modelos

- Árvore de Decisão: `DecisionTreeRegressor`
- Regressão Linear: `LinearRegression`
- Random Forest: `RandomForestRegressor`
- Gradient Boosting: `GradientBoostingRegressor`
- XGBoost: `XGBRegressor`

In [8]:
models = {
    "Decision Tree": DecisionTreeRegressor(random_state=42),
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(random_state=42),
    "XGBoost": XGBRegressor(random_state=42)
}

## Execução

In [9]:
client = MlflowClient()

# para cada semente 
for seed in seeds:
    # para cada tamanho de teste
    for test_size in test_sizes:
        # Dividir os dados em conjuntos de treino e teste
        x_train, x_test, y_train, y_test = train_test_split(x_features, y_target, test_size=test_size, random_state=seed)

        # Para cada modelo
        for model_name, model in models.items():
            # Criar o pipeline
            pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                       ('model', model)])

            # Treinar o modelo
            pipeline.fit(x_train, y_train)

            # Fazer previsões
            y_pred = pipeline.predict(x_test)

            # Avaliar o modelo
            mse = mean_squared_error(y_test, y_pred)
            r2 = r2_score(y_test, y_pred)
            mae = mean_absolute_error(y_test, y_pred)
            
            print(f"Modelo: {model_name}, Test Size: {test_size}, Seed: {seed}")
            print(f"MSE: {mse}, R2: {r2}, MAE: {mae}")
            
            # Logar os resultados no MLflow
            with mlflow.start_run():
                mlflow.log_param("model_type", model_name)
                mlflow.log_param("test_size", test_size)
                mlflow.log_param("random_seed", seed)
                
                # registrar métricas
                mlflow.log_metric("mse", mse)
                mlflow.log_metric("r2", r2)
                mlflow.log_metric("mae", mae)

                # Logar o modelo
                signature = infer_signature(x_test, y_pred)
                
                registered_model_name= model_name.lower().replace(" ", "_") + "_model"
                 
                mlflow.sklearn.log_model(
                    sk_model=pipeline,
                    artifact_path=registered_model_name,
                    signature=signature,
                    registered_model_name= registered_model_name,
                )

                #Seta todos os modelos recem treinados para Staging

                latest_versions = client.get_latest_versions(name=registered_model_name, stages=["None"])
                if latest_versions:
                    version = latest_versions[0].version
                    client.transition_model_version_stage(
                        name=registered_model_name,
                        version=version,
                        stage="Staging"
                    )
                
            mlflow.end_run()
                

Modelo: Decision Tree, Test Size: 0.1, Seed: 42
MSE: 61318563319.787544, R2: 0.4643648515387715, MAE: 173409.66728007182


Successfully registered model 'decision_tree_model'.
Created version '1' of model 'decision_tree_model'.
/tmp/ipykernel_149/2741522583.py:55: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=registered_model_name, stages=["None"])
/tmp/ipykernel_149/2741522583.py:58: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


Modelo: Linear Regression, Test Size: 0.1, Seed: 42
MSE: 53575827203.35227, R2: 0.5319998609827157, MAE: 173412.6447177572


Successfully registered model 'linear_regression_model'.
Created version '1' of model 'linear_regression_model'.
/tmp/ipykernel_149/2741522583.py:55: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=registered_model_name, stages=["None"])
/tmp/ipykernel_149/2741522583.py:58: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


Modelo: Random Forest, Test Size: 0.1, Seed: 42
MSE: 35609940151.11541, R2: 0.6889370111288469, MAE: 132206.2374586176


...

Registered model 'random_forest_model' already exists. Creating a new version of this model...
Created version '9' of model 'random_forest_model'.
/tmp/ipykernel_149/2741522583.py:55: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=registered_model_name, stages=["None"])
/tmp/ipykernel_149/2741522583.py:58: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.tra

Modelo: Gradient Boosting, Test Size: 0.3, Seed: 123
MSE: 39765471976.7416, R2: 0.6523843262185738, MAE: 148802.22336159798


Registered model 'gradient_boosting_model' already exists. Creating a new version of this model...
Created version '9' of model 'gradient_boosting_model'.
/tmp/ipykernel_149/2741522583.py:55: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=registered_model_name, stages=["None"])
/tmp/ipykernel_149/2741522583.py:58: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  cl

Modelo: XGBoost, Test Size: 0.3, Seed: 123
MSE: 35728929338.27869, R2: 0.6876703524937664, MAE: 137394.49164296337


Registered model 'xgboost_model' already exists. Creating a new version of this model...
Created version '9' of model 'xgboost_model'.
/tmp/ipykernel_149/2741522583.py:55: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=registered_model_name, stages=["None"])
/tmp/ipykernel_149/2741522583.py:58: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_mode

In [10]:
print("Treinamento concluído!")

Treinamento concluído!
